In [ ]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Game constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 950
GRID_SIZE = 8
MINE_COUNT = 5
FLAG_LIMIT = 5
TILE_SIZE = 80
MARGIN = 150
LIVES = 3  # Number of lives

# Colors
BG_COLOR = (40, 44, 52)
PANEL_COLOR = (30, 34, 42)
UNREVEALED_COLOR = (58, 95, 135)
UNREVEALED_HOVER_COLOR = (75, 115, 160)
REVEALED_COLOR = (220, 220, 225)
MINE_COLOR = (215, 95, 95)
FLAG_COLOR = (255, 50, 50)
BORDER_COLOR = (20, 20, 25)
LIFE_COLOR = (255, 80, 80)

# Number colors (classic minesweeper colors)
NUMBER_COLORS = {
    1: (45, 85, 165),
    2: (60, 145, 70),
    3: (185, 60, 60),
    4: (35, 60, 135),
    5: (150, 50, 50),
    6: (50, 140, 140),
    7: (40, 40, 40),
    8: (100, 100, 100)
}

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Minesweeper")

class Tile:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.is_mine = False
        self.is_revealed = False
        self.is_flagged = False
        self.adjacent_mines = 0
        self.hover = False

class MinesweeperGame:
    def __init__(self):
        self.grid = [[Tile(x, y) for y in range(GRID_SIZE)] for x in range(GRID_SIZE)]
        self.game_over = False
        self.game_won = False
        self.flags_placed = 0
        self.revealed_count = 0
        self.score = 0
        self.start_time = pygame.time.get_ticks()
        self.end_time = None
        self.combo = 0
        self.max_combo = 0
        self.lives = LIVES
        self.mines_hit = []  # Track which mines were hit
        self.setup_mines()
        self.calculate_adjacent_mines()
    
    def setup_mines(self):
        mines_placed = 0
        while mines_placed < MINE_COUNT:
            x = random.randint(0, GRID_SIZE - 1)
            y = random.randint(0, GRID_SIZE - 1)
            if not self.grid[x][y].is_mine:
                self.grid[x][y].is_mine = True
                mines_placed += 1
    
    def calculate_adjacent_mines(self):
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if not self.grid[x][y].is_mine:
                    count = 0
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            if dx == 0 and dy == 0:
                                continue
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                                if self.grid[nx][ny].is_mine:
                                    count += 1
                    self.grid[x][y].adjacent_mines = count
    
    def reveal_tile(self, x, y):
        if self.game_over or self.game_won:
            return
        
        tile = self.grid[x][y]
        if tile.is_revealed or tile.is_flagged:
            return
        
        tile.is_revealed = True
        self.revealed_count += 1
        
        if tile.is_mine:
            self.lives -= 1
            self.combo = 0  # Reset combo on mine hit
            self.mines_hit.append((x, y))
            
            if self.lives <= 0:
                self.game_over = True
                self.end_time = pygame.time.get_ticks()
                self.reveal_all_mines()
            return
        
        # Add score based on tile value
        if tile.adjacent_mines == 0:
            self.score += 10
            self.combo += 1
        else:
            self.score += tile.adjacent_mines * 5
            self.combo += 1
        
        # Combo bonus
        if self.combo > 5:
            self.score += self.combo * 2
        
        self.max_combo = max(self.max_combo, self.combo)
        
        if tile.adjacent_mines == 0:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                        self.reveal_tile(nx, ny)
        
        # Check win condition: all non-mine tiles revealed
        non_mine_tiles = GRID_SIZE * GRID_SIZE - MINE_COUNT
        safe_tiles_revealed = self.revealed_count - len(self.mines_hit)
        
        if safe_tiles_revealed == non_mine_tiles:
            self.game_won = True
            self.end_time = pygame.time.get_ticks()
            # Win bonus
            time_bonus = max(0, 1000 - (self.end_time - self.start_time) // 100)
            self.score += time_bonus
            # Perfect flag bonus
            if self.flags_placed == MINE_COUNT:
                self.score += 500
            # Lives bonus
            self.score += self.lives * 200
    
    def toggle_flag(self, x, y):
        if self.game_over or self.game_won:
            return
        tile = self.grid[x][y]
        if not tile.is_revealed:
            if tile.is_flagged:
                tile.is_flagged = False
                self.flags_placed -= 1
                self.combo = 0  # Reset combo on flag remove
            elif self.flags_placed < FLAG_LIMIT:
                tile.is_flagged = True
                self.flags_placed += 1
                # Bonus for correct flag
                if tile.is_mine:
                    self.score += 50
                self.combo = 0  # Reset combo on flag
    
    def reveal_all_mines(self):
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if self.grid[x][y].is_mine:
                    self.grid[x][y].is_revealed = True

def get_tile_from_mouse(mx, my):
    if my < MARGIN or my >= MARGIN + GRID_SIZE * TILE_SIZE:
        return None, None
    if mx < (SCREEN_WIDTH - GRID_SIZE * TILE_SIZE) // 2:
        return None, None
    if mx >= (SCREEN_WIDTH - GRID_SIZE * TILE_SIZE) // 2 + GRID_SIZE * TILE_SIZE:
        return None, None
    
    offset_x = (SCREEN_WIDTH - GRID_SIZE * TILE_SIZE) // 2
    grid_x = (mx - offset_x) // TILE_SIZE
    grid_y = (my - MARGIN) // TILE_SIZE
    
    if 0 <= grid_x < GRID_SIZE and 0 <= grid_y < GRID_SIZE:
        return int(grid_x), int(grid_y)
    
    return None, None

def draw_game(game):
    screen.fill(BG_COLOR)
    
    # Draw top panel
    pygame.draw.rect(screen, PANEL_COLOR, (0, 0, SCREEN_WIDTH, MARGIN))
    
    # Draw bottom panel
    pygame.draw.rect(screen, PANEL_COLOR, (0, SCREEN_HEIGHT - 80, SCREEN_WIDTH, 80))
    
    # Calculate grid offset for centering
    offset_x = (SCREEN_WIDTH - GRID_SIZE * TILE_SIZE) // 2
    offset_y = MARGIN
    
    # Draw grid
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            tile = game.grid[x][y]
            rect = pygame.Rect(offset_x + x * TILE_SIZE, offset_y + y * TILE_SIZE, TILE_SIZE, TILE_SIZE)
            
            # Draw border
            pygame.draw.rect(screen, BORDER_COLOR, rect, 3)
            
            # Shrink rect for inner tile
            inner_rect = rect.inflate(-6, -6)
            
            if tile.is_revealed:
                if tile.is_mine:
                    pygame.draw.rect(screen, MINE_COLOR, inner_rect)
                    # Draw mine circle
                    center = inner_rect.center
                    pygame.draw.circle(screen, (40, 40, 40), center, 20)
                    # Draw X if this mine was hit
                    if (x, y) in game.mines_hit:
                        pygame.draw.line(screen, (255, 255, 255), 
                                       (center[0] - 12, center[1] - 12), 
                                       (center[0] + 12, center[1] + 12), 4)
                        pygame.draw.line(screen, (255, 255, 255), 
                                       (center[0] + 12, center[1] - 12), 
                                       (center[0] - 12, center[1] + 12), 4)
                else:
                    pygame.draw.rect(screen, REVEALED_COLOR, inner_rect)
                    if tile.adjacent_mines > 0:
                        # Draw number
                        font = pygame.font.Font(None, 60)
                        text = font.render(str(tile.adjacent_mines), True, NUMBER_COLORS[tile.adjacent_mines])
                        text_rect = text.get_rect(center=inner_rect.center)
                        screen.blit(text, text_rect)
            else:
                if tile.hover:
                    pygame.draw.rect(screen, UNREVEALED_HOVER_COLOR, inner_rect)
                else:
                    pygame.draw.rect(screen, UNREVEALED_COLOR, inner_rect)
                
                if tile.is_flagged:
                    # Draw flag triangle
                    center_x, center_y = inner_rect.center
                    # Flag pole
                    pygame.draw.rect(screen, (60, 60, 60), (center_x - 2, center_y - 20, 4, 35))
                    # Flag triangle
                    flag_points = [
                        (center_x + 2, center_y - 18),
                        (center_x + 22, center_y - 8),
                        (center_x + 2, center_y + 2)
                    ]
                    pygame.draw.polygon(screen, FLAG_COLOR, flag_points)
                    pygame.draw.polygon(screen, (180, 30, 30), flag_points, 2)
    
    # Draw UI text
    font_large = pygame.font.Font(None, 48)
    font_medium = pygame.font.Font(None, 36)
    font_small = pygame.font.Font(None, 28)
    
    # Calculate time
    if game.end_time:
        elapsed = (game.end_time - game.start_time) // 1000
    else:
        elapsed = (pygame.time.get_ticks() - game.start_time) // 1000
    
    if game.game_over:
        text = font_large.render("GAME OVER!", True, (235, 100, 95))
        screen.blit(text, (SCREEN_WIDTH // 2 - text.get_width() // 2, 30))
        score_text = font_medium.render(f"Final Score: {game.score}", True, (255, 200, 100))
        screen.blit(score_text, (SCREEN_WIDTH // 2 - score_text.get_width() // 2, 80))
    elif game.game_won:
        text = font_large.render("YOU WIN!", True, (90, 200, 110))
        screen.blit(text, (SCREEN_WIDTH // 2 - text.get_width() // 2, 30))
        score_text = font_medium.render(f"Final Score: {game.score}", True, (100, 255, 150))
        screen.blit(score_text, (SCREEN_WIDTH // 2 - score_text.get_width() // 2, 80))
        
        # Show achievements
        achievements = []
        if elapsed < 60:
            achievements.append("⚡ Speed Demon!")
        if game.max_combo >= 10:
            achievements.append("🔥 Combo Master!")
        if game.flags_placed == MINE_COUNT:
            achievements.append("🎯 Perfect Flags!")
        if game.lives == LIVES:
            achievements.append("💎 Flawless Victory!")
        
        y_pos = 120
        for achievement in achievements:
            ach_text = font_small.render(achievement, True, (255, 215, 0))
            screen.blit(ach_text, (SCREEN_WIDTH // 2 - ach_text.get_width() // 2, y_pos))
            y_pos += 30
    else:
        # Top left - Stats
        text1 = font_medium.render(f"Score: {game.score}", True, (255, 215, 0))
        text2 = font_medium.render(f"Time: {elapsed}s", True, (100, 200, 255))
        screen.blit(text1, (30, 30))
        screen.blit(text2, (30, 65))
        
        # Draw lives as hearts
        heart_y = 100
        for i in range(LIVES):
            heart_x = 40 + i * 35
            if i < game.lives:
                # Filled heart
                color = LIFE_COLOR
            else:
                # Empty heart
                color = (80, 80, 80)
            
            # Draw simple heart shape
            pygame.draw.circle(screen, color, (heart_x, heart_y), 10)
            pygame.draw.circle(screen, color, (heart_x + 14, heart_y), 10)
            heart_points = [
                (heart_x - 10, heart_y + 3),
                (heart_x + 7, heart_y + 20),
                (heart_x + 24, heart_y + 3)
            ]
            pygame.draw.polygon(screen, color, heart_points)
        
        # Top right - Game info
        mines_left = MINE_COUNT - game.flags_placed
        text3 = font_medium.render(f"Mines: {mines_left}", True, (240, 200, 120))
        text4 = font_medium.render(f"Flags: {game.flags_placed}/{FLAG_LIMIT}", True, (120, 180, 230))
        screen.blit(text3, (SCREEN_WIDTH - 200, 30))
        screen.blit(text4, (SCREEN_WIDTH - 200, 65))
        
        # Combo indicator
        if game.combo > 3:
            combo_text = font_medium.render(f"Combo x{game.combo}!", True, (255, 100, 255))
            screen.blit(combo_text, (SCREEN_WIDTH // 2 - combo_text.get_width() // 2, 30))
    
    # Bottom instructions
    text1 = font_small.render("Left Click: Reveal", True, (200, 200, 200))
    text2 = font_small.render("Right Click: Flag", True, (200, 200, 200))
    text3 = font_small.render("R: Restart", True, (200, 200, 200))
    
    screen.blit(text1, (30, SCREEN_HEIGHT - 50))
    screen.blit(text2, (250, SCREEN_HEIGHT - 50))
    screen.blit(text3, (450, SCREEN_HEIGHT - 50))

def main():
    game = MinesweeperGame()
    clock = pygame.time.Clock()
    running = True
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mx, my = pygame.mouse.get_pos()
                grid_x, grid_y = get_tile_from_mouse(mx, my)
                
                if grid_x is not None:
                    if event.button == 1:  # Left click
                        game.reveal_tile(grid_x, grid_y)
                    elif event.button == 3:  # Right click
                        game.toggle_flag(grid_x, grid_y)
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    game = MinesweeperGame()
        
        # Update hover state
        mx, my = pygame.mouse.get_pos()
        hover_x, hover_y = get_tile_from_mouse(mx, my)
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                game.grid[x][y].hover = (x == hover_x and y == hover_y)
        
        draw_game(game)
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()

if __name__ == "__main__":
    main()
